In [184]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
import RAKE
import nltk
from nltk.corpus import stopwords
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
stop = stopwords.words('russian')
stop = stop + ["это", "наш", "который", "всё", "её"]
m = MorphAnalyzer()

In [182]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join([x for x in lemmas if x not in stop])

In [183]:
def normalize_rake(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

### задания 1-2.

Для корпуса я взяла пять рандомных статей с Хабра. Под статьей на Хабре указываются теги, их я и использовалась в качестве ключевых слов. Свои теги я разметила прямо в txt-документах статей. Мой корпус хранится как список кортежей, где первый элемент -- текст, второй -- мои теги, третий -- исходные теги.

In [10]:
import os
corpus = []
for filename in os.listdir('./hw1'):
    if 'txt' in filename:
        with open('./hw1/' + filename) as a:
            text = a.read()
        text = text.replace('\n', '').split('Теги: ')
        corpus.append((text[0], [normalize_text(x) for x in text[1].split(', ')], [normalize_text(x) for x in text[2].split(', ')]))

Оценим пересечение исходных тегов и тегов, проставленных мной. Я приведу количество совпадений и долю совпавших тегов (считаю ее как кол-во совпавших / объединение).

In [197]:
count_global = 0
len_global = 0
for item in corpus:
    len_keywords = len(item[1]) + len(item[2])
    count = 0
    for word in item[1]:
        if word in item[2]:
            count += 1
            len_keywords -= 1
    print(f"Количество общих keywords в документе: ", count)
    print(f"Доля общих keywords в документе: ", count/len_keywords)
    print("____________________________________")
    count_global += count
    len_global += len_keywords
print(f"Среднее количество общих keywords: ", count_global/5)
print(f"Доля общих keywords: ", count_global/len_global)

Количество общих keywords в документе:  3
Доля общих keywords в документе:  0.3
____________________________________
Количество общих keywords в документе:  4
Доля общих keywords в документе:  0.5
____________________________________
Количество общих keywords в документе:  2
Доля общих keywords в документе:  0.2
____________________________________
Количество общих keywords в документе:  3
Доля общих keywords в документе:  0.25
____________________________________
Количество общих keywords в документе:  3
Доля общих keywords в документе:  0.21428571428571427
____________________________________
Среднее количество общих keywords:  3.0
Доля общих keywords:  0.2777777777777778


Мы видим, что в среднем у нас получается три совпавших тега, доля совпадений -- 27%. Мне кажется, 3 тегов мало для интересных и насыщенных текстов с Хабра. Кроме того, я посмотрела теги Хабра, и они 1) вполне разумные, хоть и следуют немного иной логике, чем мои 2) присутствуют в текстах. Поэтому я буду использовать объединение.

In [53]:
corpus_keywords = ([(x[0], list(set(x[1]).union(x[2]))) for x in corpus])

### задание 3.


In [198]:
def get_keywords_tfidf(text):
    tf_idf_vector = vectorizer.transform([text])
    feature_array = np.array(vectorizer.get_feature_names())
    tfidf_sorting = np.argsort(tf_idf_vector.toarray()).flatten()[::-1]
    top20 = feature_array[tfidf_sorting][:20]
    return list(top20)

In [199]:
rake = RAKE.Rake(stop)
rake_res = []
for text in corpus_keywords:
    rake_text = rake.run(normalize_rake(text[0]), maxWords=3, minFrequency=2)
    rake_res.append([x[0] for x in rake_text])

In [200]:
tr_res = []
for text in corpus_keywords:
    tr_text = keywords.keywords(normalize_text(text[0]), language='russian', additional_stopwords=stop, scores=True)
    tr_res.append([x[0] for x in tr_text])

In [202]:
corpus_tfidf = [normalize_text(x[0]) for x in corpus_keywords]
vectorizer = TfidfVectorizer(stop_words=stop, smooth_idf=True, use_idf=True)
vectorizer.fit_transform(corpus_tfidf)
feature_names = vectorizer.get_feature_names()
tfidf_res = [get_keywords_tfidf(text) for text in corpus_tfidf]

### задание 4.

In [2]:
import stanza
stanza.download("ru")

2021-11-07 20:44:30 INFO: Downloading default packages for language: ru (Russian)...


2021-11-07 20:49:00 INFO: Finished downloading models and saved to /Users/veronicasmilga/stanza_resources.


Функция получения списка нужных нам шаблонов с помощью stanza

In [196]:
def get_templates(corpus_keywords):
    keywords = [x[1] for x in corpus_keywords]
    templates = []
    nlp = stanza.Pipeline("ru")
    for k in keywords:
        for word in k:
            word_tokens = [word.upos for sent in nlp(word).sentences for word in sent.words]
            templates.append(' + '.join(word_tokens))
    return set(templates)

Функция фильтрации через эти шаблоны

In [109]:
def filter_by_templates(templates, keywords):
    filtered = []
    for word in keywords:
        word_tokens = [word.upos for sent in nlp(word).sentences for word in sent.words]
        template = ' + '.join(word_tokens)
        if template in templates:
            filtered.append(word)
    return filtered

Ради интереса посмотрим на шаблоны

In [205]:
templates = get_templates(corpus_keywords)
templates

2021-11-07 22:35:05 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2021-11-07 22:35:05 INFO: Use device: cpu
2021-11-07 22:35:05 INFO: Loading: tokenize
2021-11-07 22:35:05 INFO: Loading: pos
2021-11-07 22:35:06 INFO: Loading: lemma
2021-11-07 22:35:07 INFO: Loading: depparse
2021-11-07 22:35:07 INFO: Loading: ner
2021-11-07 22:35:09 INFO: Done loading processors!


{'ADJ + NOUN',
 'NOUN',
 'NOUN + NOUN',
 'NOUN + PROPN',
 'NOUN + PUNCT + NOUN',
 'PROPN',
 'PROPN + PROPN',
 'VERB',
 'VERB + NOUN'}

Получилось вполне логично, ура. Единственный странный момент, NOUN + PUNCT + NOUN, это Сан-Франциско. Отфильтруем полученные автоматически теги с помошью шаблонов.

In [206]:
rake_res_filtered = [filter_by_templates(templates, x) for x in rake_res]
tr_res_filtered = [filter_by_templates(templates, x) for x in tr_res]
tfidf_res_filtered = [filter_by_templates(templates, x) for x in tfidf_res]

### задание 5.

Функция получения метрик

In [207]:
def get_metrics_doc(true, pred):
    pr_total = 0
    rec_total = 0
    f_total = 0
    for i in range(len(true)):
        intersection = 0
        for word in true[i]:
            if word in pred[i]:
                intersection += 1
        precision = intersection / len(pred[i])
        pr_total += intersection / len(pred[i])
        recall = intersection / len(true[i])
        rec_total += intersection / len(true[i])
        f_total += 2 * (precision * recall) / (precision + recall)
    precision = pr_total / len(true)
    recall = rec_total / len(true)
    fscore = f_total / len(true)
    return precision, recall, fscore

Функция красивого вывода метрик

In [240]:
def beautiful_metrics(no_filter, filtered, algo):
    before = get_metrics_doc([x[1] for x in corpus_keywords], no_filter)
    after = get_metrics_doc([x[1] for x in corpus_keywords], filtered)
    return (f'''Алгоритм: {algo} \nМетрики до фильтрации \nprecision: {"%.4f" % before[0]} \
recall: {"%.4f" % before[1]} fscore: {"%.4f" % before[2]} \nМетрики после фильтрации 
precision: {"%.4f" % after[0]} recall: {"%.4f" % after[1]} fscore: {"%.4f" % after[2]} 
Улучшение \nprecision: {"%.4f" % (after[0]-before[0])} recall: {"%.4f" % (after[1]-before[1])} \
fscore: {"%.4f" % (after[2]-before[2])}
''')

In [241]:
print(beautiful_metrics(rake_res, rake_res_filtered, "RAKE"))
print(beautiful_metrics(tr_res, tr_res_filtered, "TextRank"))
print(beautiful_metrics(tfidf_res, tfidf_res_filtered, "tf*idf"))

Алгоритм: RAKE 
Метрики до фильтрации 
precision: 0.3175 recall: 0.2705 fscore: 0.2670 
Метрики после фильтрации 
precision: 0.3533 recall: 0.2705 fscore: 0.2867 
Улучшение 
precision: 0.0358 recall: 0.0000 fscore: 0.0198

Алгоритм: TextRank 
Метрики до фильтрации 
precision: 0.1376 recall: 0.4593 fscore: 0.1914 
Метрики после фильтрации 
precision: 0.1885 recall: 0.4593 fscore: 0.2403 
Улучшение 
precision: 0.0509 recall: 0.0000 fscore: 0.0490

Алгоритм: tf*idf 
Метрики до фильтрации 
precision: 0.2200 recall: 0.4295 fscore: 0.2885 
Метрики после фильтрации 
precision: 0.2610 recall: 0.4295 fscore: 0.3225 
Улучшение 
precision: 0.0410 recall: 0.0000 fscore: 0.0340



### задание 6.

In [246]:
def elementswise_join(l1, l2, l3):
    result = [x + y + z for x, y, z in zip(l1, l2, l3)]
    return result

In [247]:
aut_keywords = elementswise_join(rake_res_filtered, tr_res_filtered, tfidf_res_filtered)

In [266]:
my_keywords = [x[1] for x in corpus_keywords]

In [256]:
false_negative = []
for i in range(len(my_keywords)):
    for word in my_keywords[i]:
        if word not in aut_keywords[i]:
            false_negative.append(word)

In [272]:
false_positive = []
for i in range(len(my_keywords)):
    for word in aut_keywords[i]:
        if word not in my_keywords[i]:
            false_positive.append(word)
false_positive = set(false_positive)

In [261]:
print(f"Слова, не найденные автоматически:", ', '.join(false_negative))

Слова, не найденные автоматически: пользоваться туалет, космос, возвращение, запрет, космический корабль, crew dragon, народный медицина, лечение, проблема, тренировка, забыть, запоминание, проблема, ркн, суд, rubrain, переезд, кремниевый долина, исследование


In [274]:
print(f"Лишние найденные автоматически слова:", ', '.join(false_positive))

Лишние найденные автоматически слова: находиться, основа лекарство, опыт, apple, капитализация, рубль, процесс, музей, название, узел, факт, искать факт, доступность, вид фотография, экипаж crew, хотеться, камера, представительство, турист, экскурсия музей, день рождение, использование, работа, калифорния, сок, твиттер, it, место, доработать, жизнь, понять, связанный, протокол, заметка, научный название, oracle, выехать, пройти, эффект, специалист, дело, многий, штат, говорить, работать, группа, слово, кустарник, изучение, часть, иметь, открывать, продаваться, исполнительный производство, час, город, взыскание, открыть, эмиграция, план город, мкс земля, обновление, деталь, маск, район, экипаж, онлайн, купертино, земля, элемент, улица, стоить, удалённый работа, ноябрь, inspiration, сотрудник, млн, мусор, капсула, проходить, срабатывать, стратегия, собирать, офис, информация, сожаление, twitter, прошлый год, храниться, использовать, чикаго, возбудить дело, платить, получить, проблема, зн

In [275]:
len(false_positive)

189

### Итого:
Наибольшая полнота наблюдалась у алгоритма TextRank, вероятно, засчет наибольшего количества сделанных предсказаний (на это указывает его низкая точность). Если совместить предсказания всех алгоритмов, окажется, что нам удалось найти практически бОльшую часть размеченных вручную ключевых слов (35 из 54). Однако, в то же время, даже с использованием шаблонов получилось очень много мусора (189 лишних слов). <br><br>Слова, которые не нашлись, это прежде всего слова, которые были важны для понимания статьи / связаны с ее тематикой, но встретились в статье совсем мало раз. Я смогла улучшить качество, добавив предобработку, хотя это не требовалось в задании. Предобработка помогла алгоритмам заметить те слова, которые встречались в тексте много раз, но в разных формах. <br><br>Большое количество найденного мусора связано с большим количеством шаблонов из-за разнородности проставленных мной тегов. Возможно, стоит при проставлении ограничить теги шаблонами ADJ+NOUN, NOUN, PROPN, тогда будет легче автоматически извлечь нужное. Кроме того, программа часто извлекала синонимы / контекстные синонимы тегов, которые также встречались в тексте, но ставить которые было бы нецелесообразно (получилоь бы много тегов с одинаковым значением). Может быть, можно было бы подключить БЕРТ, чтобы отсеивать очень близкие семантически слова.